## Setup

In [19]:
import pandas as pd
import numpy as np
import os
os.chdir("C:/Users/SHRADDHA RUHELA/OneDrive/Desktop/KNOWLEDGE_GRAPH")  # Set it to the project root
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

## Input data directory
data_dir = "OrfPathHealth"
inputdirectory = Path(f"./data_input/{data_dir}")
## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}")

## Load Documents

In [20]:
## Dir Loader
documents = PyPDFDirectoryLoader(inputdirectory).load()
## File Loader
# documents = PyPDFLoader("./data/MedicalDocuments/orf-path_health-n1.pdf").load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))


Number of chunks =  0


## Create a dataframe of all the chunks

In [21]:
from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(0, 0)


""


## Extract Concepts

In [22]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2ConceptsList
from helpers.df_helpers import concepts2Df

In [23]:
concepts_list = df2ConceptsList(df[10:12])

ValueError: need at least one array to concatenate

In [26]:
dfne = concepts2Df(concepts_list)
dfne.head()

,entity,importance,category,chunk_id,type
0,Mental Health,5,concept,83d4d0367bb0467e811782a4ada3bbb9,concept
1,Health Equity,4,concept,83d4d0367bb0467e811782a4ada3bbb9,concept
2,World Health Organization (WHO),3,organisation,83d4d0367bb0467e811782a4ada3bbb9,concept
3,United Nations (UN),3,organisation,83d4d0367bb0467e811782a4ada3bbb9,concept
4,Sustainable Development Goals (SDGs),4,document,83d4d0367bb0467e811782a4ada3bbb9,concept


### Write CSV to an output directory

Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of names entities

        df = dataframe of chunks

In [28]:
if not os.path.exists(outputdirectory):
   os.makedirs(outputdirectory)
   
dfne.to_csv(outputdirectory/"concepts.csv", sep="|", index=False)
df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)

## Named Entities from Concepts

**Not using this right now**

Extracting named entities our of concepts. 


In [24]:
from transformers import pipeline

ner = pipeline("token-classification", model="2rtl3/mn-xlm-roberta-base-named-entity", aggregation_strategy="simple")
# ner = pipeline("token-classification", model="dslim/bert-large-NER", aggregation_strategy="simple")

def row2NamedEntities(row):
    ner_results = ner(row['entity'])
    metadata = {'chunk_id': row['chunk_id'], 'type': 'entity'}
    entities = []
    for result in ner_results:
        entities = entities + [{'entity': result['word'], 'catetory': result['entity_group'], **metadata}]
        
    return entities



def dfText2DfNE(dataframe: pd.DataFrame):
    ## Takes a dataframe from the parsed data and returns dataframe with named entities. 
    ## The input dataframe must have a entity and a chunk_id column. 

    ## 1. Calculate named entities for each row of the dataframe. 
    results = dataframe.apply(row2NamedEntities, axis=1).reset_index(drop=True)

    ## Flatten the list of lists to one single list of entities. 
    entities_list = np.concatenate(results).ravel().tolist()

    ## Remove all NaN entities
    entities_dataframe = pd.DataFrame(entities_list).replace(' ', np.nan)
    entities_dataframe = entities_dataframe.dropna(subset=['entity'])

    ## Count the number of occurances per chunk id
    # entities_dataframe = entities_dataframe.groupby(['entity', 'category', 'chunk_id']).size().reset_index(name='count')

    return entities_dataframe

config.json:   0%|          | 0.00/934 [00:00<?, ?B/s]

C:\Users\SHRADDHA RUHELA\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SHRADDHA RUHELA\.cache\huggingface\hub\models--2rtl3--mn-xlm-roberta-base-named-entity. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [26]:

dataframe_dir = 'OrfPathHealth'  
df_concepts = pd.read_csv(f"C:/Users/SHRADDHA RUHELA/OneDrive/Desktop/knowledge_graph/data_output/MediumArticles/concepts.csv", sep="|")


dfc_split = dfText2DfNE(df_concepts)
dfc_split

,entity,catetory,chunk_id,type
0,vid,PER,c70a9f10efce49bd98ff27d64a35b5a0,entity
1,fhir,PER,d9f17a05951440e5891234fbd510f819,entity
2,world,ORG,b44d09e785ae4fc89b5bb70810e7bd99,entity
3,organization,ORG,b44d09e785ae4fc89b5bb70810e7bd99,entity
4,world,ORG,b44d09e785ae4fc89b5bb70810e7bd99,entity
...,...,...,...,...
61,the usa,LOC,2ccc33d78d5648019b9070572c8ea5d0,entity
62,australia,LOC,2ccc33d78d5648019b9070572c8ea5d0,entity
63,netherlands,LOC,2ccc33d78d5648019b9070572c8ea5d0,entity
64,nia,LOC,2ccc33d78d5648019b9070572c8ea5d0,entity
